In [71]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [72]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [73]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType

schema = StructType(fields=[
  StructField("id", StringType(), False),
  StructField("date", DateType(), False),
  StructField("date_built", DateType(), False),
  StructField("price", FloatType(), False),
  StructField("bedrooms", IntegerType(), False),
  StructField("bathrooms", IntegerType(), False),
  StructField("sqft_living", FloatType(), False),
  StructField("sqft_lot", FloatType(), False),
  StructField("floors", IntegerType(), False),
  StructField("waterfront", IntegerType(), False),
  StructField("view", IntegerType(), False)
])

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, schema=schema)

df.show()
df.describe()



24/04/22 03:17:27 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.


+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+--------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|2016-01-01|936923.0|       4|        3|     3167.0| 11733.0|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|2013-01-01|379628.0|       2|        2|     2235.0| 14384.0|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|2014-01-01|417866.0|       2|        2|     2127.0| 10575.0|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|2016-01-01|239895.0|       2|        2|     1631.0| 11149.0|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|2017-01-01|424418.0|       3|        2|     2249.0| 13878.0|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|2017-01-01|218712.0|       2|  

DataFrame[summary: string, id: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [74]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [75]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
spark.sql("""
  SELECT
    YEAR(date) as year,
    format_number(avg(price), 2) as price
  FROM home_sales
  WHERE bedrooms = 4
  GROUP BY year
  ORDER BY year DESC
""").show()


+----+----------+
|year|     price|
+----+----------+
|2022|296,363.88|
|2021|301,819.44|
|2020|298,353.78|
|2019|300,263.70|
+----+----------+



In [76]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
spark.sql("""
  SELECT
    YEAR(date_built) as year,
    format_number(avg(price), 2) as price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3
  GROUP BY year
  ORDER BY year DESC
""").show()


+----+----------+
|year|     price|
+----+----------+
|2017|292,676.79|
|2016|290,555.07|
|2015|288,770.30|
|2014|290,852.27|
|2013|295,962.27|
|2012|293,683.19|
|2011|291,117.47|
|2010|292,859.62|
+----+----------+



In [77]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
spark.sql("""
  SELECT 
    YEAR(date_built) as year,
    format_number(avg(price), 2) as price
  FROM home_sales
  WHERE bedrooms = 3 AND bathrooms = 3 AND floors >= 2 AND sqft_living >= 2000
  GROUP BY year
  ORDER BY year DESC
""").show()


+----+----------+
|year|     price|
+----+----------+
|2017|293,281.74|
|2016|300,851.65|
|2015|307,147.80|
|2014|309,821.90|
|2013|310,510.69|
|2012|319,456.36|
|2011|290,688.58|
|2010|301,130.93|
+----+----------+



In [78]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query_price_per_view = """
  SELECT view, format_number(avg(price), 2) as price
  FROM home_sales 
  WHERE price >= 350000
  GROUP BY view
  ORDER BY view DESC
"""

spark.sql(query_price_per_view).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|       price|
+----+------------+
| 100|1,026,669.50|
|  99|1,061,201.42|
|  98|1,053,739.33|
|  97|1,129,040.15|
|  96|1,017,815.92|
|  95|1,054,325.60|
|  94|1,033,536.20|
|  93|1,026,006.06|
|  92|  970,402.55|
|  91|1,137,372.73|
|  90|1,062,654.16|
|  89|1,107,839.15|
|  88|1,031,719.35|
|  87|1,072,285.20|
|  86|1,070,444.25|
|  85|1,056,336.74|
|  84|1,117,233.13|
|  83|1,033,965.93|
|  82|1,063,498.00|
|  81|1,053,472.79|
+----+------------+
only showing top 20 rows

--- 0.43862128257751465 seconds ---


The uncached query takes approximately 0.6s.

In [79]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

24/04/22 03:17:31 WARN CacheManager: Asked to cache already cached data.


DataFrame[]

In [80]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [81]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

spark.sql(query_price_per_view).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|view|       price|
+----+------------+
| 100|1,026,669.50|
|  99|1,061,201.42|
|  98|1,053,739.33|
|  97|1,129,040.15|
|  96|1,017,815.92|
|  95|1,054,325.60|
|  94|1,033,536.20|
|  93|1,026,006.06|
|  92|  970,402.55|
|  91|1,137,372.73|
|  90|1,062,654.16|
|  89|1,107,839.15|
|  88|1,031,719.35|
|  87|1,072,285.20|
|  86|1,070,444.25|
|  85|1,056,336.74|
|  84|1,117,233.13|
|  83|1,033,965.93|
|  82|1,063,498.00|
|  81|1,053,472.79|
+----+------------+
only showing top 20 rows

--- 0.16827797889709473 seconds ---


The cached query takes <0.5s

In [82]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [83]:
# 11. Read the formatted parquet data.
df_partitioned = spark.read.parquet("home_sales_partitioned")
df_partitioned.show()

+--------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|      date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+----------+--------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06|258710.0|       3|        3|     1918.0|  9666.0|     1|         0|  25|2015-01-01|
|941bad30-eb49-4a7...|2020-05-09|229896.0|       3|        3|     2197.0|  8641.0|     1|         0|   3|2015-01-01|
|c797ca12-52cd-4b1...|2019-06-08|288650.0|       2|        3|     2100.0| 10419.0|     2|         0|   7|2015-01-01|
|0cfe57f3-28c2-472...|2019-10-04|308313.0|       3|        3|     1960.0|  9453.0|     2|         0|   2|2015-01-01|
|d715f295-2fbf-4e9...|2021-05-17|391574.0|       3|        2|     1635.0|  8040.0|     2|         0|  10|2015-01-01|
|a18515a2-86f3-46b...|2022-02-18|419543.0|       3|        2|   

In [84]:
# 12. Create a temporary table for the parquet data.
df_partitioned.createOrReplaceTempView("home_sales_partitioned")

In [85]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

query_price_per_view_partitioned = query_price_per_view.replace("home_sales", "home_sales_partitioned")
spark.sql(query_price_per_view_partitioned).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|       price|
+----+------------+
| 100|1,026,669.50|
|  99|1,061,201.42|
|  98|1,053,739.33|
|  97|1,129,040.15|
|  96|1,017,815.92|
|  95|1,054,325.60|
|  94|1,033,536.20|
|  93|1,026,006.06|
|  92|  970,402.55|
|  91|1,137,372.73|
|  90|1,062,654.16|
|  89|1,107,839.15|
|  88|1,031,719.35|
|  87|1,072,285.20|
|  86|1,070,444.25|
|  85|1,056,336.74|
|  84|1,117,233.13|
|  83|1,033,965.93|
|  82|1,063,498.00|
|  81|1,053,472.79|
+----+------------+
only showing top 20 rows

--- 0.8902251720428467 seconds ---


The partitioned query takes approximately as long, if not a bit longer, than the original uncached one.

In [86]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [87]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False